### IMPORTS

In [1]:
import sys, os
sys.path.append(os.path.join(sys.path[0],'Pythons'))

In [2]:
from Carga_Descarga import *
from CRM_Central import *

### QUERIES

In [3]:
q_nr = '''WITH users_table AS (
    SELECT o.user.id AS user,
           cn.country_name AS country,
           COUNT(DISTINCT o.order_id) AS freq,
           SUM(o.shipping_amount_no_discount + amount_no_discount) AS total_amount,
           SUM(o.voucher_discount_amount) AS voucher_d,
           SUM(o.is_acquisition) AS acq
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
    WHERE o.registered_date BETWEEN DATE_TRUNC(DATE_ADD(CURRENT_DATE(),INTERVAL -1 MONTH),MONTH) AND LAST_DAY(DATE_ADD(CURRENT_DATE(),INTERVAL -1 MONTH))
          AND o.order_status = 'CONFIRMED'
    GROUP BY 1,2),
    no_return_table AS (
    SELECT ut.user AS user,
           ut.country AS country,
           ut.acq AS acq,
           SAFE_DIVIDE(ut.voucher_d,ut.total_amount) AS adr,
           ut.freq,
           CASE WHEN o.user.id IS NULL THEN TRUE ELSE FALSE END AS no_return
    FROM users_table AS ut
    LEFT JOIN `peya-bi-tools-pro.il_core.fact_orders` AS o ON ut.user = o.user.id 
                                                              AND o.registered_date BETWEEN DATE_TRUNC(CURRENT_DATE(),MONTH) AND LAST_DAY(CURRENT_DATE())
                                                              AND o.order_status = 'CONFIRMED')
SELECT nrt.country AS Country,
       IFNULL(u.city,'-') AS City,
       CASE WHEN nrt.freq = 0 THEN 'Error'
            WHEN nrt.freq <= 2 THEN '1-2'
            WHEN nrt.freq <= 4 THEN '3-4'
            WHEN nrt.freq <= 10 THEN '5-10'
            ELSE "'+10" END AS Freq_Type,
       CASE WHEN nrt.adr < 0.2 THEN 'Light ADR' ELSE 'High ADR' END AS ADR_Type,
       COUNT(DISTINCT CASE WHEN nrt.acq = 0 THEN nrt.user ELSE NULL END) AS Active_No_Acq_LM,
       COUNT(DISTINCT CASE WHEN nrt.acq = 0 AND nrt.no_return THEN nrt.user ELSE NULL END) AS No_Returns
FROM no_return_table AS nrt 
LEFT JOIN `peya-argentina.user_santiago_curat.city_usuarios` AS u ON nrt.user = u.user AND nrt.country = u.country
INNER JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON nrt.country = cn.country_name AND cn.active
GROUP BY 1,2,3,4
ORDER BY 1,2,3,4'''

q_re = '''WITH orders_table AS (
    SELECT o.registered_date AS fecha,
           cn.country_name AS country,
           o.order_id AS order_id,
           o.user.id AS user,
           CASE WHEN tc.order_id IS NOT NULL THEN TRUE ELSE FALSE END AS incentive,
           SUM(o.shipping_amount_no_discount + amount_no_discount) AS total_amount,
           SUM(o.voucher_discount_amount) AS voucher_d
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
    LEFT JOIN `peya-bi-tools-pro.il_growth.fact_talon_coupons` AS tc ON o.order_id = tc.order_id
    WHERE o.registered_date <= CURRENT_DATE()
          AND o.order_status = 'CONFIRMED'
          GROUP BY 1,2,3,4,5),
    users_table AS (
    SELECT ot.user AS user,
           ot.country AS country,
           COUNT(DISTINCT ot.order_id) AS orders,
           MAX(ot.fecha) AS last_order,
           CASE WHEN MAX(ot.fecha) <= LAST_DAY(DATE_ADD(CURRENT_DATE(),INTERVAL -2 MONTH)) THEN 'Si' ELSE 'No' END AS reactivate
    FROM orders_table AS ot
    WHERE ot.fecha <= LAST_DAY(DATE_ADD(CURRENT_DATE(),INTERVAL -1 MONTH))
    GROUP BY 1,2),
    returns_table AS (
    SELECT ot.country AS country,
           ut.user AS user,
           COUNT(DISTINCT ot.order_id) AS orders
    FROM orders_table AS ot
    INNER JOIN users_table AS ut ON ot.user = ut.user AND ot.country = ut.country
    WHERE ot.fecha BETWEEN DATE_TRUNC(CURRENT_DATE(),MONTH) AND LAST_DAY(CURRENT_DATE())
          AND ut.reactivate = 'Si'
    GROUP BY 1,2),
    incentive_table AS (
    SELECT rt.country AS country,
           rt.user AS user,
           CASE WHEN ot.incentive THEN TRUE ELSE FALSE END AS incentive
    FROM returns_table AS rt
    INNER JOIN orders_table AS ot ON rt.user = ot.user AND rt.country = ot.country
    INNER JOIN (SELECT user AS user, country AS country, MIN(order_id) AS order_id
                FROM orders_table
                WHERE fecha BETWEEN DATE_TRUNC(CURRENT_DATE(),MONTH) AND LAST_DAY(CURRENT_DATE())
                GROUP BY 1,2) AS mo ON ot.order_id = mo.order_id),
    freq_table AS (
    SELECT ot.country AS country,
           ut.user AS user,
           COUNT(DISTINCT ot.order_id) AS freq
    FROM orders_table AS ot
    INNER JOIN users_table AS ut ON ot.user = ut.user AND ot.country = ut.country
    WHERE ut.reactivate = 'Si'
          AND ot.fecha BETWEEN DATE_TRUNC(ut.last_order,MONTH) AND LAST_DAY(ut.last_order)
    GROUP BY 1,2),
    adr_table AS (
    SELECT ot.country AS country,
           ut.user AS user,
           IFNULL(SAFE_DIVIDE(SUM(ot.voucher_d),SUM(ot.total_amount)),0) AS adr
    FROM orders_table AS ot
    INNER JOIN users_table AS ut ON ot.user = ut.user AND ot.country = ut.country
    WHERE ut.reactivate = 'Si'
          AND ot.fecha BETWEEN DATE_ADD(ut.last_order,INTERVAL -181 DAY) AND LAST_DAY(ut.last_order)
    GROUP BY 1,2)
SELECT rt.country AS Country,
       IFNULL(u.city,'-') AS City,
       CASE WHEN ft.freq = 0 THEN 'Error'
            WHEN ft.freq <= 2 THEN '1-2'
            WHEN ft.freq <= 4 THEN '3-4'
            WHEN ft.freq <= 10 THEN '5-10'
            ELSE "'+10" END AS Freq_Type,
       CASE WHEN adr.adr < 0.2 THEN 'Light ADR' ELSE 'High ADR' END AS ADR_Type, 
       COUNT(DISTINCT rt.user) AS Return_Reactivated,
       COUNT(DISTINCT CASE WHEN it.incentive THEN rt.user ELSE NULL END) AS Return_Reactivated_Incentived,
       IFNULL(SUM(rt.orders),0) AS Return_Reactivated_Orders
FROM returns_table AS rt 
LEFT JOIN freq_table AS ft ON rt.user = ft.user AND rt.country = ft.country
LEFT JOIN adr_table AS adr ON rt.user = adr.user AND rt.country = adr.country
LEFT JOIN incentive_table AS it ON rt.user = it.user AND rt.country = it.country
LEFT JOIN `peya-argentina.user_santiago_curat.city_usuarios` AS u ON rt.user = u.user AND rt.country = u.country
INNER JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON rt.country = cn.country_name AND cn.active
GROUP BY 1,2,3,4
ORDER BY 1,2,3,4'''

In [4]:
# Descargo la data
bq_nr = pd.io.gbq.read_gbq(q_nr, project_id='peya-argentina', dialect='standard')

Downloading: 100%|█████████████████████████████████████████████████████████████| 3243/3243 [00:00<00:00, 4271.95rows/s]


In [5]:
bq_re = pd.io.gbq.read_gbq(q_re, project_id='peya-argentina', dialect = 'standard')

Downloading: 100%|█████████████████████████████████████████████████████████████| 2355/2355 [00:00<00:00, 3359.92rows/s]


In [6]:
# Copio las bases
nr = bq_nr.copy()
re = bq_re.copy()

### TRABAJO

In [7]:
# Trabajo No Return
val = [i for i in nr.columns if i not in ['Country','City','Freq_Type','ADR_Type']]
nr[val] = nr[val].astype(float)
col = ['Country','City','Freq_Type','ADR_Type']
nr[col] = nr[col].apply(lambda x: x.astype(str).str.upper())
# Trabajo Reactivated
val = [i for i in re.columns if i not in ['Country','City','Freq_Type','ADR_Type']]
re[val] = re[val].astype(float)
col = ['Country','City','Freq_Type','ADR_Type']
re[col] = re[col].apply(lambda x: x.astype(str).str.upper())

### CARGA

In [8]:
carga(nr,'14qn-smTGmut_pC37amCYBNRHFp9ZKzKEYCfV4YPh3-c','Crudo No Return')

Carga Correcta!


In [9]:
carga(re,'14qn-smTGmut_pC37amCYBNRHFp9ZKzKEYCfV4YPh3-c','Crudo Reactivation')

Carga Correcta!
